In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import random

from data_utils.data_utils import load_data_from_dir, AUDIO_BLOCKS

In [ ]:
from features.psd import get_psd_by_channel
from features.constants import EEG_BANDS

def get_block_features(
    blocks, subject_data, marker, channel, feature,
):
    features = []
    for b in blocks:
        block_data = subject_data[b]
        psd_data = get_psd_by_channel(block_data, marker, channel, feature)

        features = np.vstack((psd_data, features)) if len(features) > 0 else psd_data
    return features

def get_eeg_channel_feature_to_data(
    subject_data, block_list, feature_list,
):
    channel_feature_to_data = {"A": {}, "B": {}, "C": {}, "D": {}}
    for c in channel_feature_to_data.keys():
        for f in feature_list:
            raw_data = get_block_features(block_list, subject_data, "EEG", c, f)
            channel_feature_to_data[c][f.name] = raw_data

    return channel_feature_to_data

def get_user_rating_raw_labels(subject_data, blocks):
    v_labels, a_labels, attention_labels = [], [], []
    for b in blocks:
        block_data = subject_data[b]
        v_labels.extend(block_data.get_labels().flatten())
        a_labels.extend(block_data.get_labels('arousal').flatten())
        attention_labels.extend(block_data.get_labels('attention').flatten())

    return v_labels, a_labels, attention_labels

# Extract EEG spectral features

In [ ]:
data_dir = "../data/"  # Replace with your own data dir

subject_lists = []
valence_labels, arousal_labels, label_thresholds = [], [], []
marker_features = []
for i, d in enumerate(os.listdir(data_dir)):
    dir_name = data_dir + d
    if not os.path.isdir(dir_name):
        continue

    subject_data = load_data_from_dir(dir_name)
    features = get_eeg_channel_feature_to_data(
        subject_data, AUDIO_BLOCKS, EEG_BANDS.keys(),
    )
    marker_features.append(features)
    subject_lists.append(d)
    # get user rating valence and arousal
    vl, arl, _ = get_user_rating_raw_labels(subject_data, AUDIO_BLOCKS)
    valence_labels.append(vl)
    arousal_labels.append(arl)
    label_thresholds.append((np.mean(vl), np.mean(arl)))

Optional store your features/labels

In [ ]:
with open(f"result/eeg_features.pkl", "wb") as handle:
    d = {"eeg_features": marker_features}
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f"result/behavioral_labels.pkl", "wb") as handle:
    d = {
        "valence_labels": valence_labels,
        "arousal_labels": arousal_labels,
        "label_thresholds": label_thresholds,
    }
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Model training with CEBRA + KNN